<a href="https://colab.research.google.com/github/Amanda0975/MT/blob/main/Herfindahl_Hirschman_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from io import BytesIO
from zipfile import ZipFile
import pandas as pd

# Enlace del archivo en Dropbox
url = 'https://www.dropbox.com/scl/fi/ecy7lnydd0zszwvvmp8o2/data.csv.zip?dl=0&e=1&rlkey=zqyvspuxoqyxvrkr6bkjwtjik'

# Ajustar el enlace para descargar el archivo directamente
url = url.replace("?dl=0", "?dl=1")

# Descarga el archivo ZIP desde el enlace de Dropbox
response = requests.get(url)
response.raise_for_status()  # Verifica si la descarga fue exitosa

# Lee el archivo ZIP en memoria
zip_file = ZipFile(BytesIO(response.content))

# Extrae y lee el archivo CSV del archivo ZIP
with zip_file.open(zip_file.namelist()[0]) as csv_file:
    df = pd.read_csv(csv_file)


<ipython-input-1-01568c2e2492>:21: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


In [2]:
#Creación columna "Region_id"
# Definir un diccionario de mapeo para los valores de la columna "Región" a "region_id"
region_mapping = {
    'XV REGION ARICA Y PARINACOTA': 15,
    'I REGION DE TARAPACA': 1,
    'II REGION DE ANTOFAGASTA': 2,
    'III REGION DE ATACAMA': 3,
    'IV REGION COQUIMBO': 4,
    'V REGION VALPARAISO': 5,
    'XIII REGION METROPOLITANA': 13,
    'VI REGION DEL LIBERTADOR GENERAL BERNARDO O\'HIGGINS': 6,
    'VII REGION DEL MAULE': 7,
    'XVI REGION DE ÑUBLE': 16,
    'VIII REGION DEL BIO BIO': 8,
    'IX REGION DE LA ARAUCANIA': 9,
    'XIV REGION DE LOS RIOS': 14,
    'X REGION LOS LAGOS': 10,
    'XI REGION AYSEN DEL GENERAL CARLOS IBAÑEZ DEL CAMPO': 11,
    'XII REGION DE MAGALLANES Y LA ANTARTICA CHILENA': 12
}

# Aplicar el mapeo a la columna "Región" para crear la nueva columna "region_id" en el DataFrame "df"
df['region_id'] = df['Región'].map(region_mapping)

In [3]:
#Creación columna "ventas" punto medio
ventas_pm = {
    1: 0,
    2: 100,
    3: 400,
    4: 1500,
    5: 3700,
    6: 7500,
    7: 17500,
    8: 37500,
    9: 75000,
    10: 150000,
    11: 400000,
    12: 800000,
    13: 1000000
}

df['aporte'] = df['Tramo según ventas'].map(ventas_pm)

In [4]:
#Creación columnas rubro económico
rubros_unicos = df['Rubro económico'].unique()
df_1 = df[['aporte', 'Comuna']].copy()
for rubro in rubros_unicos:
      df_1[rubro] = df['aporte'] * (df['Rubro económico'] == rubro)

display(df_1)

,aporte,Comuna,Valor por Defecto,ACTIVIDADES DE ATENCION DE LA SALUD HUMANA Y DE ASISTENCIA SOCIAL,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACION DE VEHICULOS AUTOMOTORES Y MOTOCICLETAS,TRANSPORTE Y ALMACENAMIENTO,INDUSTRIA MANUFACTURERA,ACTIVIDADES DE ALOJAMIENTO Y DE SERVICIO DE COMIDAS,OTRAS ACTIVIDADES DE SERVICIOS,INFORMACION Y COMUNICACIONES,...,ACTIVIDADES FINANCIERAS Y DE SEGUROS,CONSTRUCCION,EXPLOTACION DE MINAS Y CANTERAS,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE APOYO,"SUMINISTRO DE AGUA; EVACUACION DE AGUAS RESIDUALES, GESTION DE DESECHOS Y DESCONTAMINACION",ADMINISTRACION PUBLICA Y DEFENSA; PLANES DE SEGURIDAD SOCIAL DE AFILIACION OBLIGATORIA,"ACTIVIDADES ARTISTICAS, DE ENTRETENIMIENTO Y RECREATIVAS","SUMINISTRO DE ELECTRICIDAD, GAS, VAPOR Y AIRE ACONDICIONADO",ACTIVIDADES DE ORGANIZACIONES Y ORGANOS EXTRATERRITORIALES,ACTIVIDADES DE LOS HOGARES COMO EMPLEADORES; ACTIVIDADES NO DIFERENCIADAS DE LOS HOGARES
0,100,Sin Comuna,100,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,LAS CONDES,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,400,SALAMANCA,0,0,400,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7500,OVALLE,0,0,0,7500,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1500,COQUIMBO,0,0,0,0,1500,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8361868,400,PROVIDENCIA,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8361869,1500,OSORNO,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8361870,37500,LAS CONDES,0,0,37500,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8361871,400000,SAN ANTONIO,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,400000,0,0,0


In [5]:
#Agrupación por comuna
# Obtener los valores únicos de la columna "Rubro Económico"
rubros_unicos = df['Rubro económico'].unique()

# Crear un DataFrame nuevo df_1 con las columnas "aporte" y "Comuna" de df
df_1 = df[['aporte', 'Comuna']].copy()

# Agregar columnas nuevas para cada valor único de "Rubro Económico"
for rubro in rubros_unicos:
    # Crear la nueva columna con el nombre del rubro
    df_1[rubro] = df['aporte'] * (df['Rubro económico'] == rubro)

# Sumar todas las columnas por "Comuna"
df_1 = df_1.groupby('Comuna').sum().reset_index().drop_duplicates()

# Mostrar el DataFrame resultante
display(df_1)


,Comuna,aporte,Valor por Defecto,ACTIVIDADES DE ATENCION DE LA SALUD HUMANA Y DE ASISTENCIA SOCIAL,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACION DE VEHICULOS AUTOMOTORES Y MOTOCICLETAS,TRANSPORTE Y ALMACENAMIENTO,INDUSTRIA MANUFACTURERA,ACTIVIDADES DE ALOJAMIENTO Y DE SERVICIO DE COMIDAS,OTRAS ACTIVIDADES DE SERVICIOS,INFORMACION Y COMUNICACIONES,...,ACTIVIDADES FINANCIERAS Y DE SEGUROS,CONSTRUCCION,EXPLOTACION DE MINAS Y CANTERAS,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE APOYO,"SUMINISTRO DE AGUA; EVACUACION DE AGUAS RESIDUALES, GESTION DE DESECHOS Y DESCONTAMINACION",ADMINISTRACION PUBLICA Y DEFENSA; PLANES DE SEGURIDAD SOCIAL DE AFILIACION OBLIGATORIA,"ACTIVIDADES ARTISTICAS, DE ENTRETENIMIENTO Y RECREATIVAS","SUMINISTRO DE ELECTRICIDAD, GAS, VAPOR Y AIRE ACONDICIONADO",ACTIVIDADES DE ORGANIZACIONES Y ORGANOS EXTRATERRITORIALES,ACTIVIDADES DE LOS HOGARES COMO EMPLEADORES; ACTIVIDADES NO DIFERENCIADAS DE LOS HOGARES
0,ALGARROBO,56504800,3900,364300,11033600,277500,1179300,3174200,141600,157000,...,16276700,2930400,4500,896300,84000,82700,303800,8466300,0,0
1,ALHUE,28942400,100,17900,1305300,749900,467700,313900,54100,18900,...,462600,1067700,19574800,555300,153500,76500,100,8300,0,0
2,ALTO BIO BIO,970700,15600,20500,217400,32600,9400,44900,5600,8000,...,49300,102900,0,16700,35800,0,0,66100,0,0
3,ALTO DEL CARMEN,2312500,0,0,213100,24700,343800,41100,1100,6800,...,200,184500,97700,12100,94900,4900,600,0,0,0
4,ALTO HOSPICIO,206343000,254400,297000,90295000,22491300,23998900,1641300,554200,396000,...,1718300,30824900,8114200,9675600,1061700,12000,81600,33600,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,VITACURA,4619337000,1486300,164309500,1262374700,84908700,231516200,136481100,30584200,88559300,...,798619500,436090600,39526900,158172400,13031000,59600,33143700,27579600,199400,0
343,YERBAS BUENAS,21160500,100,2500,3932200,321600,6584800,145100,10500,19800,...,12100,971600,4550000,173800,384300,0,2500,300,0,0
344,YUMBEL,19130000,6300,24200,5431800,751000,3155400,226200,546200,67600,...,388900,862600,7400,598000,137700,23600,16800,0,0,0
345,YUNGAY,29527900,1000,40300,6757600,1300200,6882800,280500,136300,61200,...,39000,1584300,750000,2628400,95800,0,3700,0,0,0


In [6]:
#Cálculo aporte por rubro económico
# Obtener los valores únicos de la columna "Rubro Económico"
rubros_unicos = df['Rubro económico'].unique()

# Crear un DataFrame nuevo df_1 con las columnas "aporte" y "Comuna" de df
df_1 = df[['aporte', 'Comuna']].copy()

# Agregar columnas nuevas para cada valor único de "Rubro Económico"
for rubro in rubros_unicos:
    # Crear la nueva columna con el nombre del rubro
    df_1[rubro] = df['aporte'] * (df['Rubro económico'] == rubro)

# Sumar todas las columnas por "Comuna"
df_1 = df_1.groupby('Comuna').sum().reset_index()

# Dividir cada columna por "aporte"
columnas_para_dividir = df_1.columns[2:]  # Seleccionar todas las columnas a partir de la tercera (después de 'aporte' y 'Comuna')
df_1[columnas_para_dividir] = df_1[columnas_para_dividir].div(df_1['aporte'], axis=0)

# Eliminar la columna 'aporte' ya que no es necesaria
df_1 = df_1.drop(columns=['aporte']).drop_duplicates()

# Mostrar el DataFrame resultante
display(df_1)


,Comuna,Valor por Defecto,ACTIVIDADES DE ATENCION DE LA SALUD HUMANA Y DE ASISTENCIA SOCIAL,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACION DE VEHICULOS AUTOMOTORES Y MOTOCICLETAS,TRANSPORTE Y ALMACENAMIENTO,INDUSTRIA MANUFACTURERA,ACTIVIDADES DE ALOJAMIENTO Y DE SERVICIO DE COMIDAS,OTRAS ACTIVIDADES DE SERVICIOS,INFORMACION Y COMUNICACIONES,ACTIVIDADES INMOBILIARIAS,...,ACTIVIDADES FINANCIERAS Y DE SEGUROS,CONSTRUCCION,EXPLOTACION DE MINAS Y CANTERAS,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE APOYO,"SUMINISTRO DE AGUA; EVACUACION DE AGUAS RESIDUALES, GESTION DE DESECHOS Y DESCONTAMINACION",ADMINISTRACION PUBLICA Y DEFENSA; PLANES DE SEGURIDAD SOCIAL DE AFILIACION OBLIGATORIA,"ACTIVIDADES ARTISTICAS, DE ENTRETENIMIENTO Y RECREATIVAS","SUMINISTRO DE ELECTRICIDAD, GAS, VAPOR Y AIRE ACONDICIONADO",ACTIVIDADES DE ORGANIZACIONES Y ORGANOS EXTRATERRITORIALES,ACTIVIDADES DE LOS HOGARES COMO EMPLEADORES; ACTIVIDADES NO DIFERENCIADAS DE LOS HOGARES
0,ALGARROBO,0.000069,0.006447,0.195268,0.004911,0.020871,0.056176,0.002506,0.002779,0.162931,...,0.288059,0.051861,0.000080,0.015862,0.001487,0.001464,0.005377,0.149833,0.000000,0.0
1,ALHUE,0.000003,0.000618,0.045100,0.025910,0.016160,0.010846,0.001869,0.000653,0.004782,...,0.015983,0.036891,0.676336,0.019186,0.005304,0.002643,0.000003,0.000287,0.000000,0.0
2,ALTO BIO BIO,0.016071,0.021119,0.223962,0.033584,0.009684,0.046255,0.005769,0.008241,0.158442,...,0.050788,0.106006,0.000000,0.017204,0.036881,0.000000,0.000000,0.068095,0.000000,0.0
3,ALTO DEL CARMEN,0.000000,0.000000,0.092151,0.010681,0.148670,0.017773,0.000476,0.002941,0.007135,...,0.000086,0.079784,0.042249,0.005232,0.041038,0.002119,0.000259,0.000000,0.000000,0.0
4,ALTO HOSPICIO,0.001233,0.001439,0.437597,0.109000,0.116306,0.007954,0.002686,0.001919,0.014774,...,0.008327,0.149387,0.039324,0.046891,0.005145,0.000058,0.000395,0.000163,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,VITACURA,0.000322,0.035570,0.273280,0.018381,0.050119,0.029546,0.006621,0.019171,0.109690,...,0.172886,0.094405,0.008557,0.034241,0.002821,0.000013,0.007175,0.005970,0.000043,0.0
343,YERBAS BUENAS,0.000005,0.000118,0.185827,0.015198,0.311184,0.006857,0.000496,0.000936,0.006129,...,0.000572,0.045916,0.215023,0.008213,0.018161,0.000000,0.000118,0.000014,0.000000,0.0
344,YUMBEL,0.000329,0.001265,0.283941,0.039258,0.164945,0.011824,0.028552,0.003534,0.025740,...,0.020329,0.045091,0.000387,0.031260,0.007198,0.001234,0.000878,0.000000,0.000000,0.0
345,YUNGAY,0.000034,0.001365,0.228855,0.044033,0.233095,0.009499,0.004616,0.002073,0.009693,...,0.001321,0.053654,0.025400,0.089014,0.003244,0.000000,0.000125,0.000000,0.000000,0.0


In [7]:
#Comprobar si columnas sumas 1
# Calcular la suma de cada fila y verificar que sea igual a 1
suma_filas = df_1.iloc[:, 1:].sum(axis=1)
assert all(abs(suma_filas - 1) < 1e-6), "¡Error! Al menos una fila no suma 1."
print("Todas las filas suman 1 correctamente.")


Todas las filas suman 1 correctamente.


In [8]:
# Suponiendo que df_1 es el DataFrame resultante después de la división y suma por "Comuna"

df_1['HHI'] = df_1.iloc[:, 1:].pow(2).sum(axis=1)
display(df_1)

,Comuna,Valor por Defecto,ACTIVIDADES DE ATENCION DE LA SALUD HUMANA Y DE ASISTENCIA SOCIAL,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACION DE VEHICULOS AUTOMOTORES Y MOTOCICLETAS,TRANSPORTE Y ALMACENAMIENTO,INDUSTRIA MANUFACTURERA,ACTIVIDADES DE ALOJAMIENTO Y DE SERVICIO DE COMIDAS,OTRAS ACTIVIDADES DE SERVICIOS,INFORMACION Y COMUNICACIONES,ACTIVIDADES INMOBILIARIAS,...,CONSTRUCCION,EXPLOTACION DE MINAS Y CANTERAS,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE APOYO,"SUMINISTRO DE AGUA; EVACUACION DE AGUAS RESIDUALES, GESTION DE DESECHOS Y DESCONTAMINACION",ADMINISTRACION PUBLICA Y DEFENSA; PLANES DE SEGURIDAD SOCIAL DE AFILIACION OBLIGATORIA,"ACTIVIDADES ARTISTICAS, DE ENTRETENIMIENTO Y RECREATIVAS","SUMINISTRO DE ELECTRICIDAD, GAS, VAPOR Y AIRE ACONDICIONADO",ACTIVIDADES DE ORGANIZACIONES Y ORGANOS EXTRATERRITORIALES,ACTIVIDADES DE LOS HOGARES COMO EMPLEADORES; ACTIVIDADES NO DIFERENCIADAS DE LOS HOGARES,HHI
0,ALGARROBO,0.000069,0.006447,0.195268,0.004911,0.020871,0.056176,0.002506,0.002779,0.162931,...,0.051861,0.000080,0.015862,0.001487,0.001464,0.005377,0.149833,0.000000,0.0,0.177251
1,ALHUE,0.000003,0.000618,0.045100,0.025910,0.016160,0.010846,0.001869,0.000653,0.004782,...,0.036891,0.676336,0.019186,0.005304,0.002643,0.000003,0.000287,0.000000,0.0,0.480871
2,ALTO BIO BIO,0.016071,0.021119,0.223962,0.033584,0.009684,0.046255,0.005769,0.008241,0.158442,...,0.106006,0.000000,0.017204,0.036881,0.000000,0.000000,0.068095,0.000000,0.0,0.118838
3,ALTO DEL CARMEN,0.000000,0.000000,0.092151,0.010681,0.148670,0.017773,0.000476,0.002941,0.007135,...,0.079784,0.042249,0.005232,0.041038,0.002119,0.000259,0.000000,0.000000,0.0,0.342750
4,ALTO HOSPICIO,0.001233,0.001439,0.437597,0.109000,0.116306,0.007954,0.002686,0.001919,0.014774,...,0.149387,0.039324,0.046891,0.005145,0.000058,0.000395,0.000163,0.000000,0.0,0.245643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
342,VITACURA,0.000322,0.035570,0.273280,0.018381,0.050119,0.029546,0.006621,0.019171,0.109690,...,0.094405,0.008557,0.034241,0.002821,0.000013,0.007175,0.005970,0.000043,0.0,0.139081
343,YERBAS BUENAS,0.000005,0.000118,0.185827,0.015198,0.311184,0.006857,0.000496,0.000936,0.006129,...,0.045916,0.215023,0.008213,0.018161,0.000000,0.000118,0.000014,0.000000,0.0,0.213389
344,YUMBEL,0.000329,0.001265,0.283941,0.039258,0.164945,0.011824,0.028552,0.003534,0.025740,...,0.045091,0.000387,0.031260,0.007198,0.001234,0.000878,0.000000,0.000000,0.0,0.203645
345,YUNGAY,0.000034,0.001365,0.228855,0.044033,0.233095,0.009499,0.004616,0.002073,0.009693,...,0.053654,0.025400,0.089014,0.003244,0.000000,0.000125,0.000000,0.000000,0.0,0.198250


In [21]:
import pandas as pd

# Suponiendo que df_HHI es tu DataFrame existente

# Verificar si hay valores distintos de HHI para la misma comuna pero diferente año comercial
comunas_valores_diferentes_HHI = df_HHI.groupby('Comuna')['HHI'].nunique() > 1

# Filtrar las comunas con valores diferentes de HHI
comunas_con_valores_diferentes_HHI = comunas_valores_diferentes_HHI[comunas_valores_diferentes_HHI]

# Mostrar las comunas con valores diferentes de HHI
print(df_HHI.loc[df_HHI['Comuna'].isin(comunas_con_valores_diferentes_HHI.index), 'Comuna'])


Series([], Name: Comuna, dtype: object)


In [10]:
#Merge ambas dataframe
data_HHI = pd.merge(df, df_1, on="Comuna")

In [11]:
#Datos año 2021 y 2022
data_HHI = data_HHI[data_HHI['Año comercial'].isin([2021, 2022])].drop_duplicates()
data_HHI = data_HHI[data_HHI['Tipo de contribuyente'].isin(['PERSONA JURIDICA COMERCIAL'])].drop_duplicates()

In [22]:
# Suponiendo que data_HHI es el DataFrame original

# Columnas que queremos mantener en el nuevo DataFrame
columnas_filtradas = [
    "Año comercial", "Comuna", "region_id",
    "ACTIVIDADES DE ATENCION DE LA SALUD HUMANA Y DE ASISTENCIA SOCIAL",
    "COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACION DE VEHICULOS AUTOMOTORES Y MOTOCICLETAS",
    "TRANSPORTE Y ALMACENAMIENTO",
    "INDUSTRIA MANUFACTURERA",
    "ACTIVIDADES DE ALOJAMIENTO Y DE SERVICIO DE COMIDAS",
    "OTRAS ACTIVIDADES DE SERVICIOS",
    "INFORMACION Y COMUNICACIONES",
    "ACTIVIDADES INMOBILIARIAS",
    "AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA",
    "ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS",
    "ENSEÑANZA",
    "ACTIVIDADES FINANCIERAS Y DE SEGUROS",
    "CONSTRUCCION",
    "EXPLOTACION DE MINAS Y CANTERAS",
    "ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE APOYO",
    "SUMINISTRO DE AGUA; EVACUACION DE AGUAS RESIDUALES, GESTION DE DESECHOS Y DESCONTAMINACION",
    "ADMINISTRACION PUBLICA Y DEFENSA; PLANES DE SEGURIDAD SOCIAL DE AFILIACION OBLIGATORIA",
    "ACTIVIDADES ARTISTICAS, DE ENTRETENIMIENTO Y RECREATIVAS",
    "SUMINISTRO DE ELECTRICIDAD, GAS, VAPOR Y AIRE ACONDICIONADO",
    "ACTIVIDADES DE ORGANIZACIONES Y ORGANOS EXTRATERRITORIALES",
    "ACTIVIDADES DE LOS HOGARES COMO EMPLEADORES; ACTIVIDADES NO DIFERENCIADAS DE LOS HOGARES",
    "HHI"
]

# Crear el nuevo DataFrame df_HHI
df_HHI = data_HHI[columnas_filtradas].copy().drop_duplicates()

In [23]:
display(df_HHI)

,Año comercial,Comuna,region_id,ACTIVIDADES DE ATENCION DE LA SALUD HUMANA Y DE ASISTENCIA SOCIAL,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACION DE VEHICULOS AUTOMOTORES Y MOTOCICLETAS,TRANSPORTE Y ALMACENAMIENTO,INDUSTRIA MANUFACTURERA,ACTIVIDADES DE ALOJAMIENTO Y DE SERVICIO DE COMIDAS,OTRAS ACTIVIDADES DE SERVICIOS,INFORMACION Y COMUNICACIONES,...,CONSTRUCCION,EXPLOTACION DE MINAS Y CANTERAS,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE APOYO,"SUMINISTRO DE AGUA; EVACUACION DE AGUAS RESIDUALES, GESTION DE DESECHOS Y DESCONTAMINACION",ADMINISTRACION PUBLICA Y DEFENSA; PLANES DE SEGURIDAD SOCIAL DE AFILIACION OBLIGATORIA,"ACTIVIDADES ARTISTICAS, DE ENTRETENIMIENTO Y RECREATIVAS","SUMINISTRO DE ELECTRICIDAD, GAS, VAPOR Y AIRE ACONDICIONADO",ACTIVIDADES DE ORGANIZACIONES Y ORGANOS EXTRATERRITORIALES,ACTIVIDADES DE LOS HOGARES COMO EMPLEADORES; ACTIVIDADES NO DIFERENCIADAS DE LOS HOGARES,HHI
5136,2021,Sin Comuna,NaN,0.000550,0.014731,0.002007,0.001668,0.012431,0.000543,0.000819,...,0.002742,0.003641,0.004839,0.000090,0.000000,0.000058,0.009895,0.000000,0.000000e+00,0.712569
5507,2022,Sin Comuna,NaN,0.000550,0.014731,0.002007,0.001668,0.012431,0.000543,0.000819,...,0.002742,0.003641,0.004839,0.000090,0.000000,0.000058,0.009895,0.000000,0.000000e+00,0.712569
701506,2021,LAS CONDES,13.0,0.014519,0.154694,0.044170,0.048057,0.015390,0.006076,0.031651,...,0.112496,0.026046,0.046688,0.004135,0.000385,0.002338,0.032606,0.000028,1.741303e-07,0.135948
773110,2022,LAS CONDES,13.0,0.014519,0.154694,0.044170,0.048057,0.015390,0.006076,0.031651,...,0.112496,0.026046,0.046688,0.004135,0.000385,0.002338,0.032606,0.000028,1.741303e-07,0.135948
855695,2021,SALAMANCA,4.0,0.004916,0.127279,0.194081,0.019720,0.049198,0.034689,0.006263,...,0.193185,0.020030,0.082635,0.015344,0.000363,0.000970,0.000188,0.000000,0.000000e+00,0.126170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8360124,2022,ISLA DE PASCUA,5.0,0.001050,0.015296,0.016453,0.002556,0.017184,0.003357,0.001847,...,0.014029,0.000000,0.010192,0.000061,0.000000,0.000677,0.010233,0.000000,0.000000e+00,0.448701
8361322,2021,HUALAIHUE,10.0,0.001241,0.424477,0.167889,0.071101,0.018108,0.000022,0.004122,...,0.066934,0.000355,0.047186,0.001485,0.000022,0.000310,0.000000,0.000000,0.000000e+00,0.250925
8361570,2022,HUALAIHUE,10.0,0.001241,0.424477,0.167889,0.071101,0.018108,0.000022,0.004122,...,0.066934,0.000355,0.047186,0.001485,0.000022,0.000310,0.000000,0.000000,0.000000e+00,0.250925
8361868,2021,ANTARTIDA,12.0,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,...,0.930108,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.866979


#Exportar

In [24]:
# Crear dos DataFrames separados basados en el valor de "Año comercial"
df_HHI_1 = df_HHI[df_HHI['Año comercial'] == 2022]

# Guardar cada DataFrame en un archivo CSV
df_HHI_1.to_csv('HHI_2021_2022.csv', index=False)

#Descripción

In [19]:
data_HHI[['Comuna', 'Año comercial', 'HHI']].sort_values(by='Comuna').drop_duplicates()

,Comuna,Año comercial,HHI
7169742,ALGARROBO,2022,0.177251
7169421,ALGARROBO,2021,0.177251
7883632,ALHUE,2022,0.480871
7883484,ALHUE,2021,0.480871
8354374,ALTO BIO BIO,2021,0.118838
...,...,...,...
6645534,YUMBEL,2022,0.203645
8157987,YUNGAY,2022,0.198250
8157725,YUNGAY,2021,0.198250
8238983,ZAPALLAR,2021,0.543201


In [ ]:
data_HHI.shape


(1646735, 47)

In [ ]:
data_HHI['Tipo de contribuyente'].unique()

array(['SIN PER. JURIDICA', 'ORG. SIN FINES DE LUCRO',
       'PERSONA JURIDICA COMERCIAL', 'SOCIEDADES EXTRANJERAS',
       'INSTITUCIONES FISCALES', 'MUNICIPALIDADES',
       'ORGANISMOS INTERNACIONALES'], dtype=object)

In [ ]:
df['Rubro económico'].unique()

array(['Valor por Defecto',
       'ACTIVIDADES DE ATENCION DE LA SALUD HUMANA Y DE ASISTENCIA SOCIAL',
       'COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACION DE VEHICULOS AUTOMOTORES Y MOTOCICLETAS',
       'TRANSPORTE Y ALMACENAMIENTO', 'INDUSTRIA MANUFACTURERA',
       'ACTIVIDADES DE ALOJAMIENTO Y DE SERVICIO DE COMIDAS',
       'OTRAS ACTIVIDADES DE SERVICIOS', 'INFORMACION Y COMUNICACIONES',
       'ACTIVIDADES INMOBILIARIAS',
       'AGRICULTURA, GANADERIA, SILVICULTURA Y PESCA',
       'ACTIVIDADES PROFESIONALES, CIENTIFICAS Y TECNICAS', 'ENSEÑANZA',
       'ACTIVIDADES FINANCIERAS Y DE SEGUROS', 'CONSTRUCCION',
       'EXPLOTACION DE MINAS Y CANTERAS',
       'ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE APOYO',
       'SUMINISTRO DE AGUA; EVACUACION DE AGUAS RESIDUALES, GESTION DE DESECHOS Y DESCONTAMINACION',
       'ADMINISTRACION PUBLICA Y DEFENSA; PLANES DE SEGURIDAD SOCIAL DE AFILIACION OBLIGATORIA',
       'ACTIVIDADES ARTISTICAS, DE ENTRETENIMIENTO Y RECREATIVAS',
 

In [ ]:
df.describe()

,Año comercial,RUT,Tramo según ventas,Número de trabajadores dependie,Tramo capital propio positivo,Tramo capital propio negativo
count,8.361873e+06,8.361873e+06,8.361873e+06,8.361873e+06,5.005932e+06,1.333494e+06
mean,2.015671e+03,7.704621e+07,3.784698e+00,1.648113e+01,6.869126e+00,5.775029e+00
std,4.993435e+00,6.983203e+06,2.467218e+00,2.224188e+02,2.569356e+00,2.678571e+00
min,2.005000e+03,4.700301e+07,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,2.012000e+03,7.621020e+07,2.000000e+00,0.000000e+00,5.000000e+00,4.000000e+00
50%,2.017000e+03,7.664224e+07,4.000000e+00,0.000000e+00,7.000000e+00,6.000000e+00
75%,2.020000e+03,7.735231e+07,5.000000e+00,3.000000e+00,9.000000e+00,8.000000e+00
max,2.022000e+03,9.959986e+07,1.300000e+01,6.387400e+04,1.000000e+01,1.000000e+01


In [ ]:
from IPython.display import display
display(df)

,Año comercial,RUT,DV,Razón social,Tramo según ventas,Número de trabajadores dependie,Fecha inicio de actividades vige,Fecha término de giro,Fecha primera inscripción de ac,Tipo término de giro,...,Tramo capital propio positivo,Tramo capital propio negativo,Rubro económico,Subrubro económico,Actividad económica,Región,Provincia,Comuna,R_PRESUNTA,OTROS_REGIMENES
0,2005,47012566,7,BBVA CORREDORA DE BOLSA LIMITADA,2,0,NaN,NaN,NaN,NaN,...,NaN,NaN,Valor por Defecto,Valor por Defecto,Valor por Defecto,Sin Información,Sin provincia,Sin Comuna,NaN,NaN
1,2005,50000100,3,FUNDACION ARTURO IRARRAZAVAL CORREA,1,6,1993-01-01,NaN,1993-01-01,NaN,...,NaN,NaN,ACTIVIDADES DE ATENCION DE LA SALUD HUMANA Y D...,OTRAS ACTIVIDADES DE ATENCION EN INSTITUCIONES,OTRAS ACTIVIDADES DE ATENCION EN INSTITUCIONES,XIII REGION METROPOLITANA,Santiago,LAS CONDES,NaN,NaN
2,2005,50000430,4,SAAVEDRA GALLARDO JAVIER Y HERMANOS,3,0,1993-01-01,NaN,1993-01-01,NaN,...,NaN,NaN,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,"VENTA AL POR MENOR DE ALIMENTOS, BEBIDAS Y TAB...",VENTA AL POR MENOR DE ALIMENTOS EN COMERCIOS E...,IV REGION COQUIMBO,Choapa,SALAMANCA,NaN,NaN
3,2005,50000510,6,TORREJON CASTRO LUIS Y JOSE,6,0,1993-01-01,NaN,1993-01-01,NaN,...,9.0,NaN,TRANSPORTE Y ALMACENAMIENTO,OTRAS ACTIVIDADES DE TRANSPORTE POR VIA TERRESTRE,TRANSPORTE DE CARGA POR CARRETERA,IV REGION COQUIMBO,Limarí,OVALLE,NaN,NaN
4,2005,50000710,9,PONCE PONCE MANUEL E HIJOS,4,0,1993-01-01,NaN,1993-01-01,NaN,...,5.0,NaN,INDUSTRIA MANUFACTURERA,FABRICACION DE PRODUCTOS METALICOS PARA USO ES...,FABRICACION DE PRODUCTOS METALICOS PARA USO ES...,IV REGION COQUIMBO,Elqui,COQUIMBO,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8361868,2022,99599570,0,INSTITUTO DE BELLEZA INTEGRAL DE PELUQUERIA S.A.,3,4,2005-06-30,NaN,2005-06-30,NaN,...,NaN,9.0,ENSEÑANZA,OTROS TIPOS DE ENSEÑANZA,OTROS TIPOS DE ENSEÑANZA N.C.P.,XIII REGION METROPOLITANA,Santiago,PROVIDENCIA,NaN,REGIMEN GENERAL SEMI INTEGRADO (14A)
8361869,2022,99599690,1,INDOSTRIAS AMERICANAS DI MODENA S.A.,4,0,2005-07-01,NaN,2005-07-01,NaN,...,6.0,NaN,ACTIVIDADES INMOBILIARIAS,ACTIVIDADES INMOBILIARIAS REALIZADAS CON BIENE...,"COMPRA, VENTA Y ALQUILER (EXCEPTO AMOBLADOS) D...",X REGION LOS LAGOS,Osorno,OSORNO,NaN,REGIMEN GENERAL SEMI INTEGRADO (14A)
8361870,2022,99599740,1,FERRAGAMO CHILE S.A.,8,11,2005-07-01,NaN,2005-07-01,NaN,...,10.0,NaN,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...,VENTA AL POR MAYOR DE ENSERES DOMESTICOS,"VENTA AL POR MAYOR DE ARTICULOS DE PERFUMERIA,...",XIII REGION METROPOLITANA,Santiago,LAS CONDES,NaN,REGIMEN GENERAL SEMI INTEGRADO (14A)
8361871,2022,99599760,6,CASINO DE JUEGOS DEL PACIFICO S.A.,11,272,2005-07-01,NaN,2005-07-01,NaN,...,10.0,NaN,"ACTIVIDADES ARTISTICAS, DE ENTRETENIMIENTO Y R...",ACTIVIDADES DE JUEGOS DE AZAR Y APUESTAS,ACTIVIDADES DE CASINOS DE JUEGOS,V REGION VALPARAISO,San Antonio,SAN ANTONIO,NaN,REGIMEN GENERAL SEMI INTEGRADO (14A)


In [ ]:
df['Región'].unique()

array(['XIII REGION METROPOLITANA', 'IV REGION COQUIMBO',
       'II REGION DE ANTOFAGASTA', 'X REGION LOS LAGOS',
       'V REGION VALPARAISO', 'VII REGION DEL MAULE',
       'VIII REGION DEL BIO BIO',
       "VI REGION DEL LIBERTADOR GENERAL BERNARDO O'HIGGINS",
       'XVI REGION DE ÑUBLE', 'XV REGION ARICA Y PARINACOTA',
       'III REGION DE ATACAMA', 'XIV REGION DE LOS RIOS',
       'IX REGION DE LA ARAUCANIA',
       'XII REGION DE MAGALLANES Y LA ANTARTICA CHILENA',
       'I REGION DE TARAPACA',
       'XI REGION AYSEN DEL GENERAL CARLOS IBAÑEZ DEL CAMPO',
       'Sin Información'], dtype=object)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8361873 entries, 0 to 8361872
Data columns (total 22 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   Año comercial                     int64  
 1   RUT                               int64  
 2   DV                                object 
 3   Razón social                      object 
 4   Tramo según ventas                int64  
 5   Número de trabajadores dependie   int64  
 6   Fecha inicio de actividades vige  object 
 7   Fecha término de giro             object 
 8   Fecha primera inscripción de ac   object 
 9   Tipo término de giro              object 
 10  Tipo de contribuyente             object 
 11  Subtipo de contribuyente          object 
 12  Tramo capital propio positivo     float64
 13  Tramo capital propio negativo     float64
 14  Rubro económico                   object 
 15  Subrubro económico                object 
 16  Actividad económica               ob

In [ ]:
df.nunique()

Año comercial                            18
RUT                                 1437661
DV                                       11
Razón social                        1382221
Tramo según ventas                       13
Número de trabajadores dependie        6092
Fecha inicio de actividades vige      10564
Fecha término de giro                  5576
Fecha primera inscripción de ac        9156
Tipo término de giro                      4
Tipo de contribuyente                     7
Subtipo de contribuyente                 52
Tramo capital propio positivo            10
Tramo capital propio negativo            10
Rubro económico                          22
Subrubro económico                      235
Actividad económica                     669
Región                                   17
Provincia                                57
Comuna                                  347
R_PRESUNTA                                1
OTROS_REGIMENES                           8
dtype: int64